# Text Generation Pattern

## Text Generation with zero shot prompt

### Invoking a Model 

The code above will invoke a model for text generation with zero prompt




In [1]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


### Generate text


In [3]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

### Building de body for query

In [4]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

### Invoking the Model
First explore how model generates output based on the prompt created earlier



In [5]:
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [6]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print(email)

Subject: Apology for the inconvenience caused

Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the inconvenience caused by the service provided by our customer support engineer.

We understand that your experience with us was not up to the mark, and we take your feedback seriously. We have thoroughly investigated the matter and taken steps to prevent such incidents from happening in the future.

I would like to assure you that we are committed to providing excellent customer service and resolving any issues you may have. Our team will do everything they can to assist you and ensure that your experience with us is positive.

Please accept our apologies once again for the inconvenience caused. If you have any further concerns or questions, please do not hesitate to contact us.

Sincerely,
Bob
Customer Service Manager


### Streaming the output

In [7]:
output = []
try:
    
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

		**Chunk 1**

Subject: Apology for the inconvenience caused

Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the inconvenience caused by the service provided by our 

		**Chunk 2**
customer support engineer.

We understand that your experience with us was not up to the mark, and we take your feedback seriously. We have thoroughly investigated the matter and taken steps to prevent such incidents from happening in the future.

I would like to assure you that we are committed to providing excellent customer service and resolving any issues

		**Chunk 3**
 you may have. Our team will do everything they can to assist you and ensure that your experience with us is positive.

Please accept our apologies once again for the inconvenience caused. If you have any further concerns or questions, please do not hesitate to contact us.

Sincerely,
Bob
Customer Service Manager



In [8]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

		**COMPLETE OUTPUT**


Subject: Apology for the inconvenience caused

Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the inconvenience caused by the service provided by our customer support engineer.

We understand that your experience with us was not up to the mark, and we take your feedback seriously. We have thoroughly investigated the matter and taken steps to prevent such incidents from happening in the future.

I would like to assure you that we are committed to providing excellent customer service and resolving any issues you may have. Our team will do everything they can to assist you and ensure that your experience with us is positive.

Please accept our apologies once again for the inconvenience caused. If you have any further concerns or questions, please do not hesitate to contact us.

Sincerely,
Bob
Customer Service Manager


## With a prompt template

### Invoking the Bedrock LLM Model using Langchain



In [11]:
#%pip install langchain-aws
from langchain_aws import ChatBedrock as Bedrock
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'

inference_modifier = { 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

chat = Bedrock(model_id = modelId, model_kwargs = inference_modifier)

chat.invoke("Helo!")

AIMessage(content='Hello!', additional_kwargs={'usage': {'prompt_tokens': 11, 'completion_tokens': 4, 'total_tokens': 15}}, response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 11, 'completion_tokens': 4, 'total_tokens': 15}}, id='run-dfb7238c-4883-4e37-b4e2-f9cd76d914ba-0')

### Create a LangChain Custom Prompt
Now we will create a langchain custom prompt template with different variables

In [17]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [18]:
#%pip install anthropic
num_tokens = chat.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

Our prompt has 127 tokens


### Invoke Again
Now we will invoke the model with the customized prompt

In [19]:
response = chat.invoke(prompt)

print(response.content)

Dear John Doe,

Please accept my sincere apologies for the unsatisfactory experience you had with our customer support. We strive to provide excellent service, and it is clear that we fell short of meeting your expectations.

I want to assure you that the delay in responding promptly and the incorrect initial suggestion are not up to our standards. We will thoroughly investigate the circumstances that led to this situation and take appropriate measures to prevent such occurrences in the future.

Your satisfaction is our top priority, and we value your business greatly. I understand your frustration and the potential consideration of taking your business elsewhere. However, I hope that you will give us another opportunity to regain your trust and demonstrate our commitment to delivering outstanding service.

Please feel free to reach out to me directly if you have any further concerns or if there is anything else we can do to make this right. We are determined to learn from this experie